In [2]:
using Plots
using StaticArrays
import LinearAlgebra
import FastGaussQuadrature

push!(LOAD_PATH, pwd())
include("./physconsts.jl")
using .PhysConst


In [1]:
include("RTtypes.jl")

abstract type AbstractSGrid end

# using .RTypes


mutable struct CartGrid
    """ Physical space grid from simulation """
    
    x1 :: Matrix{Float64}

end

mutable struct CylGrid
    """ Cylindrical space grid from simulation """
    
    x1 :: Matrix{Float64}

end

mutable struct TGrid
    """ Tangent space grid from simulation """
    
    x1 :: Matrix{Float64}

end



# make TGrid from SGrid